# Unicorn Companies
A unicorn company is a privately held company with a current valuation of over $1 billion USD. This dataset consists of unicorn companies and startups across the globe as of November 2021, including country of origin, sector, select investors, and valuation of each unicorn. 

_Note former unicorn companies that have since exited due to IPO or acquisitions are not included in this list._

Not sure where to begin? Scroll to the bottom to find challenges!

**SCENERIO QUESTION**

You have been hired as a data scientist for a company that invests in start-ups. Your manager is interested in whether it is possible to predict whether a company reaches a valuation over 5 billion based on characteristics such as its country of origin, its category, and details about its investors.

Using the dataset provided, you have been asked to test whether such predictions are possible, and the confidence one can have in the results.

You will need to prepare a report that is accessible to a broad audience. It will need to outline your motivation, steps, findings, and conclusions.

In [4]:
SELECT *
FROM industries;

,company_id,industry
0,189,Health
1,848,Fintech
2,556,Internet software & services
3,999,Internet software & services
4,396,Fintech
...,...,...
1069,1002,Mobile & telecommunications
1070,215,E-commerce & direct-to-consumer
1071,866,Internet software & services
1072,651,Fintech


In [11]:
-- This code examines the performance of each industry per country

SELECT
    i.industry,
    c.country,
    c.company,
    valuation,
    funding
FROM
    companies c
JOIN
    industries i ON c.company_id = i.company_id
JOIN
    funding f ON i.company_id = f.company_id
GROUP BY
    i.industry,
    c.country,
    c.company,
    valuation,
    funding
ORDER BY
    valuation DESC;

,industry,country,company,valuation,funding
0,Artificial intelligence,China,Bytedance,180000000000,8000000000
1,E-commerce & direct-to-consumer,China,SHEIN,100000000000,2000000000
2,Other,United States,SpaceX,100000000000,7000000000
3,Fintech,United States,Stripe,95000000000,2000000000
4,Fintech,Sweden,Klarna,46000000000,4000000000
...,...,...,...,...,...
1069,Travel,Germany,GetYourGuide,1000000000,656000000
1070,Travel,Germany,Omio,1000000000,396000000
1071,Travel,Hong Kong,Klook,1000000000,722000000
1072,Travel,Spain,TravelPerk,1000000000,408000000


[Source](https://www.cbinsights.com/research-unicorn-companies) of dataset. 

In [10]:
SELECT
    c.company,
    i.industry,
    c.country,
    f.valuation
FROM
    companies c
JOIN
    industries i ON c.company_id = i.company_id
JOIN
    funding f ON i.company_id = f.company_id
GROUP BY
    i.industry, c.company, c.country, f.valuation
ORDER BY
    f.valuation DESC;


,company,industry,country,valuation
0,Bytedance,Artificial intelligence,China,180000000000
1,SHEIN,E-commerce & direct-to-consumer,China,100000000000
2,SpaceX,Other,United States,100000000000
3,Stripe,Fintech,United States,95000000000
4,Klarna,Fintech,Sweden,46000000000
...,...,...,...,...
1069,Klook,Travel,Hong Kong,1000000000
1070,Omio,Travel,Germany,1000000000
1071,TravelPerk,Travel,Spain,1000000000
1072,VOI,Travel,Sweden,1000000000


![image-7](image-7.png)


In [4]:
WITH country_index AS (
    SELECT
        industry,
        country,
        company,
        valuation,
        funding,
        ROUND(AVG(valuation), 2) avg_valuation,
        ROUND(AVG(funding), 2) Average_funding,
        MAX(valuation) max_valuation,
        MIN(valuation) min_valuation,
        SUM(valuation) country_industry_val,
        MAX(funding) max_funding, 
        MIN(funding) min_funding,
        SUM(funding) industry_gross_funding
    FROM
        companies c
    JOIN
        industries i ON c.company_id = i.company_id
    JOIN
        funding f ON i.company_id = f.company_id
    GROUP BY
        country, industry, company, valuation, funding
    ORDER BY
        avg_valuation DESC
),
global_index AS (
    SELECT
        industry,
        COUNT(i.company_id),
        ROUND(AVG(valuation), 2) global_avg_valuation,
        SUM(valuation) tot_global_val
    FROM
        industries i
    JOIN
        funding f ON i.company_id = f.company_id
    GROUP BY
        industry
    ORDER BY
        global_avg_valuation DESC
)
SELECT *
FROM
    country_index c
JOIN
    global_index g USING (industry);


,industry,country,company,valuation,funding,avg_valuation,average_funding,max_valuation,min_valuation,country_industry_val,max_funding,min_funding,industry_gross_funding,count,global_avg_valuation,tot_global_val
0,Artificial intelligence,China,Bytedance,180000000000,8000000000,1.800000e+11,8.000000e+09,180000000000,180000000000,180000000000,8000000000,8000000000,8000000000,84,4.488095e+09,377000000000
1,Other,United States,SpaceX,100000000000,7000000000,1.000000e+11,7.000000e+09,100000000000,100000000000,100000000000,7000000000,7000000000,7000000000,58,4.344828e+09,252000000000
2,E-commerce & direct-to-consumer,China,SHEIN,100000000000,2000000000,1.000000e+11,2.000000e+09,100000000000,100000000000,100000000000,2000000000,2000000000,2000000000,111,3.837838e+09,426000000000
3,Fintech,United States,Stripe,95000000000,2000000000,9.500000e+10,2.000000e+09,95000000000,95000000000,95000000000,2000000000,2000000000,2000000000,224,3.937500e+09,882000000000
4,Fintech,Sweden,Klarna,46000000000,4000000000,4.600000e+10,4.000000e+09,46000000000,46000000000,46000000000,4000000000,4000000000,4000000000,224,3.937500e+09,882000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,"Supply chain, logistics, & delivery",India,BlackBuck,1000000000,352000000,1.000000e+09,3.520000e+08,1000000000,1000000000,1000000000,352000000,352000000,352000000,57,3.105263e+09,177000000000
1070,Fintech,United States,Ibotta,1000000000,93000000,1.000000e+09,9.300000e+07,1000000000,1000000000,1000000000,93000000,93000000,93000000,224,3.937500e+09,882000000000
1071,Cybersecurity,United States,Noname Security,1000000000,220000000,1.000000e+09,2.200000e+08,1000000000,1000000000,1000000000,220000000,220000000,220000000,50,2.580000e+09,129000000000
1072,Other,Japan,Spiber,1000000000,520000000,1.000000e+09,5.200000e+08,1000000000,1000000000,1000000000,520000000,520000000,520000000,58,4.344828e+09,252000000000


When I ranked each country by its percentage of average global valuation, countries with only one company were mostly returned leaving out the ones with more companies but with higher total valuation. 
Hence I used the total valuation for each indusrtry per country and how it compares with the global valuation. 
I used function RANK_FILTER.* to filter my ranking return only the top 3 positions across all industries.

In [9]:
WITH country_index AS (
    SELECT  industry,
            country,
            COUNT (c.company_id) no_of_companies,
            ROUND(AVG(valuation),2) avg_valuation,
            ROUND(AVG(funding),2) average_funding,
            MAX(valuation) max_valuation,
            MIN(valuation) min_valuation,
            SUM(valuation) country_industry_val,
            MAX(funding) max_funding, 
            MIN(funding) min_funding,
            SUM(funding) industry_total_funding
    FROM companies c
    JOIN industries i ON c.company_id = i.company_id
    JOIN funding f ON i.company_id = f.company_id
    GROUP BY 2,1
),
global_index AS (
    SELECT industry,
            COUNT(i.company_id),
            ROUND(AVG(valuation),2) global_avg_valuation,
            SUM(valuation) tot_global_val
    FROM industries i
    JOIN funding f ON i.company_id = f.company_id
    GROUP BY 1
)

SELECT c.industry,
       c.country,
       c.no_of_companies,
       c.avg_valuation,
       c.country_industry_val,
       g.tot_global_val,
       g.global_avg_valuation, 
       ROUND((c.avg_valuation * 100) / g.global_avg_valuation, 2) AS perc_global_avg_val,
       RANK() OVER (PARTITION BY c.industry ORDER BY (c.avg_valuation * 100) / g.global_avg_valuation DESC) AS industry_rank
FROM country_index c
JOIN global_index g ON c.industry = g.industry
WHERE c.no_of_companies >= 3 AND c.avg_valuation > g.global_avg_valuation
ORDER BY perc_global_avg_val DESC;


,industry,country,no_of_companies,avg_valuation,country_industry_val,tot_global_val,global_avg_valuation,perc_global_avg_val,industry_rank
0,Internet software & services,Australia,5,9.200000e+09,46000000000,595000000000,2.902439e+09,316.97,1
1,Artificial intelligence,China,20,1.070000e+10,214000000000,377000000000,4.488095e+09,238.41,1
2,E-commerce & direct-to-consumer,China,29,6.448276e+09,187000000000,426000000000,3.837838e+09,168.02,1
3,Other,United States,30,6.500000e+09,195000000000,252000000000,4.344828e+09,149.60,1
4,Edtech,India,6,5.166667e+09,31000000000,100000000000,3.571429e+09,144.67,1
5,Fintech,United Kingdom,26,5.653846e+09,147000000000,882000000000,3.937500e+09,143.59,1
6,Cybersecurity,Canada,3,3.333333e+09,10000000000,129000000000,2.580000e+09,129.20,1
7,Mobile & telecommunications,United States,12,3.000000e+09,36000000000,89000000000,2.342105e+09,128.09,1
8,"Supply chain, logistics, & delivery",United States,20,3.950000e+09,79000000000,177000000000,3.105263e+09,127.20,1
9,Consumer & retail,United States,11,5.363636e+09,59000000000,106000000000,4.240000e+09,126.50,1


From the above query,I ranked the industries per country based on the average valuation compared to the global average. This will help identify the actual performance of industries where we have at least 3 companies. From the previous ranking USA & CHINA dominated the top 20 having 95% of the biggest industries.

Meanwhile, using average industry valuation, other countries like (AUSTRALIA,INDIA,CANADA,UNITED KINGDOM) came into the list ranking first in (Internet software & services, Edtech, Cybersecurity, and Fintech) respectively.

![image-11](image-11.png)


In [11]:
-- List the 20 most valued companies
SELECT
	c.company,
	f.valuation,
	country,
	industry
FROM companies c
JOIN funding f
ON c.company_id =f.company_id
JOIN industries i
ON f.company_id = i.company_id
GROUP BY c.company,f.valuation,country,industry
ORDER BY valuation DESC
LIMIT 20;

,company,valuation,country,industry
0,Bytedance,180000000000,China,Artificial intelligence
1,SpaceX,100000000000,United States,Other
2,SHEIN,100000000000,China,E-commerce & direct-to-consumer
3,Stripe,95000000000,United States,Fintech
4,Klarna,46000000000,Sweden,Fintech
5,Canva,40000000000,Australia,Internet software & services
6,Checkout.com,40000000000,United Kingdom,Fintech
7,Instacart,39000000000,United States,"Supply chain, logistics, & delivery"
8,JUUL Labs,38000000000,United States,Consumer & retail
9,Databricks,38000000000,United States,Data management & analytics


In [12]:
--Next, we calculate our measures of central tendency to understand the spread of valuation across all industries.

SELECT
	industry,
    AVG(valuation),
    MAX(valuation),
    MIN(valuation),	
	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY valuation) AS median,
	PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY valuation) AS third_percentile,
	PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY valuation) AS first_percentile,
	STDDEV(valuation)
FROM industries i
JOIN funding f ON i.company_id = f.company_id
GROUP BY industry
ORDER BY AVG(valuation) DESC;

,industry,avg,max,min,median,third_percentile,first_percentile,stddev
0,Artificial intelligence,4.488095e+09,180000000000,1000000000,2000000000,2250000000,1000000000,19512426716
1,Other,4.344828e+09,100000000000,1000000000,2000000000,2000000000,1000000000,13520032861
2,Consumer & retail,4.240000e+09,38000000000,1000000000,2000000000,2000000000,1000000000,7833049640
3,Fintech,3.937500e+09,95000000000,1000000000,2000000000,3000000000,1000000000,8263357172
4,E-commerce & direct-to-consumer,3.837838e+09,100000000000,1000000000,2000000000,3000000000,1000000000,9985936056
5,Edtech,3.571429e+09,22000000000,1000000000,2000000000,3000000000,1000000000,4932346534
6,Data management & analytics,3.317073e+09,38000000000,1000000000,2000000000,3000000000,1000000000,5884042082
7,Travel,3.285714e+09,10000000000,1000000000,2000000000,4500000000,1000000000,3172685624
8,Auto & transportation,3.193548e+09,11000000000,1000000000,2000000000,4000000000,1000000000,2880038829
9,"Supply chain, logistics, & delivery",3.105263e+09,39000000000,1000000000,1000000000,3000000000,1000000000,5780769260


In [13]:
--This code refines our last query to give more details of industries in each country.

SELECT
	industry,
	COUNT(i.company_id) AS company_count,
	c.country,
    AVG(f.valuation),
    MAX(f.valuation),
    MIN(f.valuation),	
	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY f.valuation) AS median,
	PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY f.valuation) AS third_percentile,
	PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY f.valuation) AS first_percentile,
	STDDEV(f.valuation)
FROM industries i
JOIN funding f ON i.company_id = f.company_id
JOIN companies c ON i.company_id = c.company_id
GROUP BY industry, c.country
ORDER BY AVG(f.valuation) DESC;

,industry,company_count,country,avg,max,min,median,third_percentile,first_percentile,stddev
0,Fintech,1,Sweden,4.600000e+10,46000000000,46000000000,46000000000,46000000000,46000000000,NaN
1,Fintech,1,Bahamas,3.200000e+10,32000000000,32000000000,32000000000,32000000000,32000000000,NaN
2,"Supply chain, logistics, & delivery",1,Indonesia,2.000000e+10,20000000000,20000000000,20000000000,20000000000,20000000000,NaN
3,Other,1,Sweden,1.200000e+10,12000000000,12000000000,12000000000,12000000000,12000000000,NaN
4,E-commerce & direct-to-consumer,1,Turkey,1.200000e+10,12000000000,12000000000,12000000000,12000000000,12000000000,NaN
...,...,...,...,...,...,...,...,...,...,...
189,Data management & analytics,1,Switzerland,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN
190,Mobile & telecommunications,1,United Kingdom,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN
191,Data management & analytics,1,Israel,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN
192,Other,1,Canada,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN


In [19]:
-- This code removes the outlier company from Artificial intelligence
-- and then returns the industry average valuation

SELECT 
	i.industry AS industry_name, 
	AVG(f.valuation) AS industry_avg_valuation
FROM funding f
JOIN companies c ON c.company_id = f.company_id 
JOIN industries i ON f.company_id = i.company_id 
WHERE 
    c.company NOT IN ('Bytedance', 'SpaceX', 'Epic Games')

GROUP BY i.industry
ORDER BY industry_avg_valuation DESC;


,industry_name,industry_avg_valuation
0,Artificial intelligence,4.488095e+09
1,Other,4.344828e+09
2,Consumer & retail,4.240000e+09
3,Fintech,3.937500e+09
4,E-commerce & direct-to-consumer,3.837838e+09
5,Edtech,3.571429e+09
6,Data management & analytics,3.317073e+09
7,Travel,3.285714e+09
8,Auto & transportation,3.193548e+09
9,"Supply chain, logistics, & delivery",3.105263e+09


The graphs below uses measures of central tendency to visualize the distribution of company valuation across the four industries to have a feel of valuation data spread. 

![image-3](image-3.png)
![image-4](image-4.png)
![image-5](image-5.png)
![image-6](image-6.png)


Artificial intelligence industry recorded the highest average valuation among the Unicorn companies. It is observed that the valuation for artificial intelligence has an outlier effect from Bytedance a China based company with a valuation of 180 Billion USD which impacted significantly on the industry average. The distribution is positively skewed,with median at 2 billion USD which implies that half of the artificial intelligence companies have a valuation of $2Billion.

Removing the outlier from Artificial intelligence industries we have the mean drop to 2.37 billion USD which is closer to the median. Similarly we filtered 'Other' category to remove two outliers (SpaceX and Epic Games) which also saw the mean drop from 4.3 billion USD to 2.14 billion USD. 

The top 20 performing companies in Artificial intelligence are situated in the USA and CHINA. 
Fifteen(14) companies are from USA while four(4) are CHINESE based and only one each from SINGAPORE and UNITED KINGDOM.

We also have an interesting scenerio in the Fintech industry with USA having the an outlier company worth 95 billion USD, while the industry average is less than 5 billion USD. United Kindom has the highest total valuation in Fintech with its industry average at 5.6 billion  USD and a total of 26 unicorn companies. 



In [2]:
/* Code to extract the names of investors in the Unicorn companies */

WITH sub1 AS (
    SELECT
        company,
        country,
        valuation,
        funding,
        select_investors,
        SPLIT_PART(select_investors, ',', 1) AS Investor1,
        SPLIT_PART(select_investors, ',', 2) AS Investor2,
        SPLIT_PART(select_investors, ',', 3) AS Investor3,
        SPLIT_PART(select_investors, ',', 4) AS Investor4
    FROM (
        SELECT
            company,
            country,
            valuation,
            select_investors,
            funding,
            RANK() OVER (ORDER BY valuation DESC) AS company_rank
        FROM
            companies c
        JOIN
            funding f ON c.company_id = f.company_id
        JOIN
            industries i ON f.company_id = i.company_id
        WHERE
            valuation >= 5000000000
    ) AS Ranking
    ORDER BY valuation DESC
)

SELECT
    company,
    valuation,
    investor1
FROM
    sub1
UNION  
SELECT
    company,
    valuation,
    investor2
FROM
    sub1
UNION 
SELECT
    company,
    valuation,
    investor3
FROM
    sub1
UNION 
SELECT
    company,
    valuation,
    investor4
FROM
    sub1
ORDER BY valuation DESC;


,company,valuation,investor1
0,Bytedance,180000000000,Softbank Group
1,Bytedance,180000000000,Sequoia Capital China
2,Bytedance,180000000000,Sina Weibo
3,Bytedance,180000000000,SIG Asia Investments
4,SpaceX,100000000000,Draper Fisher Jurvetson
...,...,...,...
644,Creditas,5000000000,Amadeus Capital Partners
645,C6 Bank,5000000000,Credit Suisse
646,BetterUp,5000000000,Lightspeed Venture Partners
647,Pleo,5000000000,


In [7]:

/*I used aggregate function with case statment and partition by to group the companies into less than or above $5 billion valuation
 then I used CTE with row number to filter for the industry per country*/
  
  SELECT *
  FROM 
   ( With cte AS 
(SELECT Industry,c.country,
COUNT (i.company_id) OVER (PARTITION  BY country, industry) tot_company, 
SUM(CASE WHEN valuation >= 5000000000 THEN 1 ELSE  0 END ) OVER (PARTITION BY country, industry ) above_5B, 
SUM (CASE WHEN valuation < 5000000000 THEN 1 ELSE 0 END ) OVER (PARTITION BY country, industry) less_5B
 
FROM industries i
JOIN funding f
ON i.company_id =f.company_id
JOIN companies c
ON f.company_id = c.company_id) 

SELECT industry, 
    country,
    tot_company, 
above_5B/tot_company :: NUMERIC AS probability, 
ROW_NUMBER() OVER (PARTITION BY country,industry ORDER BY country  )  
FROM cte) prob3
WHERE row_number BETWEEN 0 and 1

    

,industry,country,tot_company,probability,row_number
0,Fintech,Argentina,1,0.000000,1
1,E-commerce & direct-to-consumer,Australia,1,0.000000,1
2,Fintech,Australia,2,0.500000,1
3,Internet software & services,Australia,5,0.200000,1
4,Edtech,Austria,1,0.000000,1
...,...,...,...,...,...
189,Other,United States,30,0.133333,1
190,"Supply chain, logistics, & delivery",United States,20,0.100000,1
191,Travel,United States,1,1.000000,1
192,Fintech,Vietnam,1,0.000000,1


![image-13](image-13.png)


In [5]:

  /* To get an insight from the investors in companies with valuation over $5B, I have sorted and used split function for the last query, next the columns are collapse together using subquery and then union the table to itself selecting each investor column each time and then  can count the investors.*/

 
WITH ranked_companies AS (
    SELECT
        company,
        country,
        valuation,
        funding,
        select_investors,
        SPLIT_PART(select_investors, ',', 1) AS Investor1,
        SPLIT_PART(select_investors, ',', 2) AS Investor2,
        SPLIT_PART(select_investors, ',', 3) AS Investor3,
        SPLIT_PART(select_investors, ',', 4) AS Investor4,
        RANK() OVER (ORDER BY valuation DESC) AS company_rank
    FROM
        companies c
    JOIN
        funding f ON c.company_id = f.company_id
    JOIN
        industries i ON f.company_id = i.company_id
    WHERE
        valuation >= 5000000000
)
SELECT
    company,
    valuation,
    investor
FROM (
    SELECT company, valuation, Investor1 AS investor, company_rank FROM ranked_companies
    UNION ALL
    SELECT company, valuation, Investor2 AS investor, company_rank FROM ranked_companies
    UNION ALL
    SELECT company, valuation, Investor3 AS investor, company_rank FROM ranked_companies
    UNION ALL
    SELECT company, valuation, Investor4 AS investor, company_rank FROM ranked_companies
) AS combined_data
ORDER BY company_rank, valuation DESC;


,company,valuation,investor
0,Bytedance,180000000000,Sina Weibo
1,Bytedance,180000000000,Softbank Group
2,Bytedance,180000000000,Sequoia Capital China
3,Bytedance,180000000000,SIG Asia Investments
4,SHEIN,100000000000,Tiger Global Management
...,...,...,...
667,DailyHunt,5000000000,Falcon Edge Capital
668,UBTECH Robotics,5000000000,CDH Investments
669,OneTrust,5000000000,Insight Partners
670,Cockroach Labs,5000000000,


In [9]:
SELECT
    company,
    valuation,
    industry,
    c.country,
    ROUND(AVG(valuation) OVER (PARTITION BY country ORDER BY industry), 2) AS avg_valuation,
    SUM(valuation) OVER (PARTITION BY country ORDER BY industry) AS running_sum,
    MAX(valuation) OVER (PARTITION BY country ORDER BY industry) AS max_valuation
FROM
    industries i
JOIN
    funding f ON i.company_id = f.company_id
JOIN
    companies c ON f.company_id = c.company_id;


,company,valuation,industry,country,avg_valuation,running_sum,max_valuation,min_valuation
0,Uala,2000000000,Fintech,Argentina,2.000000e+09,2000000000,2000000000,2000000000
1,Pet Circle,1000000000,E-commerce & direct-to-consumer,Australia,1.000000e+09,1000000000,1000000000,1000000000
2,Immutable,3000000000,Fintech,Australia,3.333333e+09,10000000000,6000000000,1000000000
3,Airwallex,6000000000,Fintech,Australia,3.333333e+09,10000000000,6000000000,1000000000
4,Canva,40000000000,Internet software & services,Australia,7.000000e+09,56000000000,40000000000,1000000000
...,...,...,...,...,...,...,...,...
1069,Loadsmart,1000000000,"Supply chain, logistics, & delivery",United States,3.433155e+09,1926000000000,100000000000,1000000000
1070,STORD,1000000000,"Supply chain, logistics, & delivery",United States,3.433155e+09,1926000000000,100000000000,1000000000
1071,TripActions,7000000000,Travel,United States,3.439502e+09,1933000000000,100000000000,1000000000
1072,MoMo,2000000000,Fintech,Vietnam,2.000000e+09,2000000000,2000000000,2000000000


In [8]:
/* This code uses case statement to filter the conditions for counting the number of
industries with less than $5 bllion valuation based on date joined and year founded. */




SELECT
    COUNT(f.company_id) AS count_company,
    year_founded,
    CASE WHEN valuation >= 5000000000 THEN '1' ELSE '0' END AS condition
FROM
    funding f
JOIN
    dates d ON f.company_id = d.company_id
GROUP BY
    year_founded,
    condition
ORDER BY
    year_founded DESC;



,count_company,year_founded,condition
0,11,2021,0
1,3,2020,1
2,22,2020,0
3,42,2019,0
4,3,2019,1
5,10,2018,1
6,51,2018,0
7,10,2017,1
8,64,2017,0
9,98,2016,0


Now we have the list of companies with more than $5B valuation and also the countries with most valued industry. 
We also have the list of investors with average valuation of companies invested and count of the companies.

The companies that joined the unicorn league from year 2014 - 2020 have higher probability of achieving up to $5billion valuation. The probability ranges between 14% - 30% based on the chart below, comparing their percentages to total listed company for those years.

There is a growth in the number of companies founded between 2008 - 2018 which also showed a corresponding increase in the number of those with valuation of at least $5 billion.

The best investment option will be made in countries where the industry is within the top 3   industries as shown in the charts. Also the year joined and year founded is another insight to guide our investment options. Finally, looking at number of the investor, there is no much risk as over 200 investors have invested in companies with average of over $5 billion. 

![image-9](image-9.png)
![image-10](image-10.png)
